## <font color='blue'>Linear regression on California housing data</font>

The California Housing data set was obtained from the 1990 California census. One use of it is to predict housing prices based on features such as house age, location, number of bedrooms, etc.

In the data, the housing has been divided into "blocks", each a geographically compact area containing on average 1400 individuals. There are 20,640 data points, one per block.

Each data point has the following information about the corresponding block:
* median income (multiples of 10K) in that block
* median house age
* average number of rooms in housing in that block
* average number of bedrooms
* population
* average occupancy of houses in block
* latitude
* longitude
* median house value (multiples of 100K)
The regression problem is to predict the house value based on the other 8 features.

### <font color='blue'>1. Loading the data and getting some summary statistics</font>

In addition to `numpy` and `matplotlib` we will be using `pandas`. This gives us a handy way of storing the data in "frames" which include attribute names.

In [1]:
import pandas as pd
import numpy as np

Now let's load in the data and take a quick look at it. The display has one point per row. Notice how nice the formatting is, and how each column is named according to its feature.

In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing(as_frame=True)
df = housing.frame  # a Pandas data-frame
display(df)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


Now let's look at the correlations between these 9 variables. We can use the `corr()` method in `pandas` for this, and then display the resulting matrix using some nice formatting.

In [3]:
# Compute correlation matrix
corr_matrix = df.corr() 
# Print it nicely
corr_matrix.style \
    .background_gradient(cmap='coolwarm') \
    .format(precision=2)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
MedInc,1.00,-0.12,0.33,-0.06,0.00,0.02,-0.08,-0.02,0.69
HouseAge,-0.12,1.00,-0.15,-0.08,-0.30,0.01,0.01,-0.11,0.11
AveRooms,0.33,-0.15,1.00,0.85,-0.07,-0.00,0.11,-0.03,0.15
AveBedrms,-0.06,-0.08,0.85,1.00,-0.07,-0.01,0.07,0.01,-0.05
Population,0.00,-0.30,-0.07,-0.07,1.00,0.07,-0.11,0.10,-0.02
AveOccup,0.02,0.01,-0.00,-0.01,0.07,1.00,0.00,0.00,-0.02
Latitude,-0.08,0.01,0.11,0.07,-0.11,0.00,1.00,-0.92,-0.14
Longitude,-0.02,-0.11,-0.03,0.01,0.10,0.00,-0.92,1.00,-0.05
MedHouseVal,0.69,0.11,0.15,-0.05,-0.02,-0.02,-0.14,-0.05,1.00


<font color='magenta'>Some questions for you:</font>
* Which (other) feature is most highly correlated with median house value?
* Which pair of features are the most strongly correlated?
* Which pair of features are the most negatively correlated?

### <font color='blue'>2. The regression problem</font>

Next, we'll separate the predictor variables (the first eight columns) from the response variable (the last column). 

We will also split the data into training and test set. There is a nice method for this in `sklearn.model_selection`.

In [4]:
from sklearn.model_selection import train_test_split

# Separate predictor variables (X) from response (y)
X = df.drop(columns=['MedHouseVal'])  # Features
y = df['MedHouseVal']                 # Target

# Split data into training set (X_train, y_train) and test set (X_test, y_test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<font color='magenta'>Some questions for you:</font>
* What are the sizes of the training and test sets?
* Suppose we want to predict `y` (house value) without seeing `x`; what value of `y` would work best for the test set, and what would be the resulting mean squared error on the test set?

In [18]:
print(X_train.shape, X_test.shape)

mean_y_test = y_test.mean()

print(mean_y_test)

mse_y_test = ((y_test - mean_y_test)**2).mean()
print(mse_y_test)

(16512, 8) (4128, 8)
2.0550030959302323
1.3104089782408996


Answer:
1. The size of the training set is 16512*8, and the size of the test set is 4128*8.
2. We will use the mean value of all the y_test data, which is 2.06, and the mean square error is 1.31.

To do: Use `sklearn.linear_model.LinearRegression` to fit a linear function to the training data using least-squares regression. Then display the resulting coefficients of each of the 8 features and give the mean squared error on the test set.<font color='magenta'> 

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Initialize the linear regression model
model = LinearRegression()

# Train the data
model.fit(X_train, y_train)

LinearRegression()

In [20]:
# Display the resulting coefficients of each of the 8 features
for feature, coef in zip(X_train.columns, model.coef_):
    print(f"{feature}: {coef}")

MedInc: 0.44867490966571855
HouseAge: 0.009724257517905635
AveRooms: -0.12332334282795901
AveBedrms: 0.7831449067929722
Population: -2.029620580143027e-06
AveOccup: -0.003526318487134158
Latitude: -0.419792486588358
Longitude: -0.4337080649639871


In [21]:
# Obtain the prediction of the X_test data
y_pred = model.predict(X_test)

# Calculate the MSE
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on test set:", mse)

Mean Squared Error on test set: 0.5558915986952444


<font color='magenta'> To do: Again, we'll fit a linear function (using the training set) and get the mean squared error (on the test set). However, this time we will use just a subset of the features.</font>
* Use just the two features `Latitude` and `Longitude`
* Use just one feature; which is the best choice?

 In both cases, report the resulting mean squared error on the test set.

In [24]:
# Choose the features: Latitude and Longitude
X_train_LatLong = X_train[['Latitude', 'Longitude']]
X_test_LatLong = X_test[['Latitude', 'Longitude']]

# Fit the linear regression model
model_LatLong = LinearRegression()
model_LatLong.fit(X_train_LatLong, y_train)

# Predict in the sub test set
y_pred_LatLong = model_LatLong.predict(X_test_LatLong)

# Calculate MSE
mse_LatLong = mean_squared_error(y_test, y_pred_LatLong)
print("MSE using Latitude and Longitude:", mse_LatLong)

MSE using Latitude and Longitude: 0.9788435787714895


In [25]:
# Store all the features
feature_mse = {}

for feature in X_train.columns:
    # Only use one feature
    X_train_single = X_train[[feature]]
    X_test_single = X_test[[feature]]
    
    # Fit the linear regression model
    model_single = LinearRegression()
    model_single.fit(X_train_single, y_train)
    
    # Predict the result
    y_pred_single = model_single.predict(X_test_single)
    
    # Calculate MSE
    mse_single = mean_squared_error(y_test, y_pred_single)
    feature_mse[feature] = mse_single

# Find the featue which has the prediction with the minimum MSE
best_feature = min(feature_mse, key=feature_mse.get)
best_mse = feature_mse[best_feature]

print(f"Best single feature: {best_feature}")
print(f"MSE using only {best_feature}:", best_mse)

Best single feature: MedInc
MSE using only MedInc: 0.7091157771765548
